### Трегубович Е.И., m3206
#### вариант 4
# Лабораторная работа №5

###Построение модели

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

Для построения линейной модели воспользуемся методом наименьших квадратов. Разделим данные на зависимую переменную (популярность) и независимые переменные (продолжительность, танцевальность и энергичность). Добавим столбец единиц к матрице X для учёта свободного коэффициента:

In [ ]:
data = pd.read_csv('song_data.csv')
X = data[['song_duration_ms', 'danceability', 'energy']]
y = data['song_popularity']
X = np.column_stack((np.ones(len(X)), X))

 Вычислим оценки коэффициентов модели с помощью формулы:
 $ \hat{\beta} = (X^TX)^{-1}X^Ty $. Вычислим остаточную дисперсию как среднеквадратичное отклонение остатков.







In [ ]:
beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y
y_hat = X @ beta_hat
residuals = y - y_hat
mse = np.sum(residuals**2) / (len(y) - len(beta_hat))

Строим доверительные интервалы для коэффициентов модели с помощью t-статистики:

In [ ]:
se = np.sqrt(np.diag(mse * np.linalg.inv(X.T @ X)))
t = stats.t.ppf(0.975, len(y) - len(beta_hat))
ci = np.column_stack((beta_hat - t*se, beta_hat + t*se))

Вычисляем коэффициент детерминации как отношение объяснённой дисперсии к общей дисперсии:

In [ ]:
ssr = np.sum((y_hat - np.mean(y))**2)
sst = np.sum((y - np.mean(y))**2)
r2 = ssr / sst
print(r2)

0.01094657082334394


#### Проверка гипотез

1. $H_0$: $\beta_3$ = 0 (энергичность не влияет на популярность) \\
$H_A$: $\beta_3$ > 0 (энергичность положительно связана с популярностью)

Если p-значение меньше уровня значимости 0,05, то отвергаем нулевую гипотезу в пользу альтернативной:

In [ ]:
t_stat = beta_hat[3] / se[3]
p_val = 1 - stats.t.cdf(t_stat, len(y) - len(beta_hat))
alpha = 0.05
if p_val < alpha:
    print('Отвергаем нулевую гипотезу. Энергия положительно влияет на популярность.')
else:
    print('Не отвергаем нулевую гипотезу.')

Не отвергаем нулевую гипотезу.


2. $H_0$: $\beta_1$ = 0 (продолжительность не влияет на популярность) \\
$H_A$: $\beta_1$ ≠ 0 (продолжительность связана с популярностью)

Если p-значение меньше уровня значимости 0,05/2, то отвергаем нулевую гипотезу в пользу альтернативной:

In [ ]:
t_stat = beta_hat[1] / se[1]
p_val = 2 * (1 - stats.t.cdf(abs(t_stat), len(y) - len(beta_hat)))
alpha = 0.05/2
if p_val < alpha:
    print('Отвергаем нулевую гипотезу. Продолжительность связана с популярностью.')
else:
    print('Не отвергаем нулевую гипотезу.')

Не отвергаем нулевую гипотезу.


3. $H_0$: $\beta_2$ = $\beta_3$ = 0 (танцевальность и энергичность не влияют на популярность) \\
$H_A$: хотя бы один из коэффициентов $\beta_2$ и $\beta_3$ ≠ 0


Если p-значение меньше уровня значимости 0,05, то отвергаем нулевую гипотезу в пользу альтернативной:

In [ ]:
F_stat = (ssr / 2) / (mse * (len(y) - len(beta_hat) - 2))
p_val = 1 - stats.f.cdf(F_stat, 2, len(y) - len(beta_hat) - 2)
alpha = 0.05
if p_val < alpha:
    print('Отвергаем нулевую гипотезу. Nанцевальность и энергичность влияют на популярность.')
else:
    print('Не отвергаем нулевую гипотезу.')

Не отвергаем нулевую гипотезу.


# На основе полученных результатов можно сделать следующие выводы:

- Энергичность не связана с популярностью песни.
- Продолжительность песни не оказывает статистически значимого влияния на её популярность.
- Танцевальность и энергичность песни не оказывают статистически значимое влияние на её популярность.

Коэффициент детерминации довольно низкий, что говорит о том, что модель не объясняет большую часть вариации в данных.